In [1]:
import os
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

urlretrieve('http://www.wanghao.in/data/ctrsr_datasets.rar', 'ctrsr_datasets.rar')
os.system('unrar x ctrsr_datasets.rar')

32512

In [3]:
import numpy as np
import random

users_count = 0
interactions_count = 0
with open('ctrsr_datasets/citeulike-a/users.dat', 'r') as fin:
    for line in fin:
        #print(line)
        interactions_count += int(line.split()[0])
        users_count += 1
        
print('users_count',users_count)
print('interactions_count',interactions_count)# sum of interactions

users_count 5551
interactions_count 204986


In [4]:
# randomly hold out an item per user for validation and testing respectively.
val_structured_arr = np.zeros(users_count, dtype=[('user_id', np.int32), ('item_id', np.int32)]) 
test_structured_arr = np.zeros(users_count, dtype=[('user_id', np.int32), ('item_id', np.int32)])
train_structured_arr = np.zeros(interactions_count-users_count*2, dtype=[('user_id', np.int32), ('item_id', np.int32)])

In [15]:
interaction_ind = 0
next_user_id = 0
next_item_id = 0
map_to_item_id = dict()  # Map: item--id from 0 to len(items)-1

with open('ctrsr_datasets/citeulike-a/users.dat', 'r') as fin:
    for line in fin:
        item_list = line.split()[1:]
        random.shuffle(item_list)
        for ind, item in enumerate(item_list):
            #print((item_list))
           # print(ind, item)
            if item not in map_to_item_id:
                map_to_item_id[item] = next_item_id
                next_item_id += 1
            if ind == 0:
                val_structured_arr[next_user_id] = (next_user_id, map_to_item_id[item])
            elif ind == 1:
                test_structured_arr[next_user_id] = (next_user_id, map_to_item_id[item])
            else:
                train_structured_arr[interaction_ind] = (next_user_id, map_to_item_id[item])
                interaction_ind += 1
        next_user_id += 1

In [25]:

from openrec.utils import ImplicitDataset

train_dataset = ImplicitDataset(raw_data=train_structured_arr, 
                        max_user=users_count, 
                        max_item=len(map_to_item_id), name='Train')
val_dataset = ImplicitDataset(raw_data=val_structured_arr, 
                      max_user=users_count,
                      max_item=len(map_to_item_id), name='Val')
test_dataset = ImplicitDataset(raw_data=test_structured_arr, 
                       max_user=users_count,
                       max_item=len(map_to_item_id), name='Test')

In [28]:
from openrec.recommenders import BPR

bpr_model = BPR(batch_size=1000, 
                max_user=train_dataset.max_user(), 
                max_item=train_dataset.max_item(), 
                dim_embed=20, 
                opt='Adam')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [29]:
from openrec.utils.samplers import PairwiseSampler

sampler = PairwiseSampler(batch_size=1000, dataset=train_dataset, num_process=1)

In [30]:
from openrec.utils.evaluators import AUC

auc_evaluator = AUC()

In [31]:

from openrec import ImplicitModelTrainer

model_trainer = ImplicitModelTrainer(batch_size=1000, 
                             test_batch_size=100, 
                            train_dataset=train_dataset, 
                             model=bpr_model, 
                             sampler=sampler)

In [32]:
model_trainer.train(num_itr=10000, 
                    display_itr=1000, 
                    eval_datasets=[val_dataset, test_dataset],
                    evaluators=[auc_evaluator])

== Start training with FULL evaluation ==
[Itr 1000] loss: 574.027180
..(dataset: Val) evaluation


100%|██████████| 56/56 [00:46<00:00,  1.21it/s]

..(dataset: Val) AUC 0.8679326481883544
..(dataset: Test) evaluation



100%|██████████| 56/56 [00:46<00:00,  1.20it/s]


..(dataset: Test) AUC 0.8655168683044783
[Itr 2000] loss: 285.036109
..(dataset: Val) evaluation


...

..(dataset: Test) AUC 0.9479867046362126
[Itr 9000] loss: 57.103045
..(dataset: Val) evaluation


100%|██████████| 56/56 [00:45<00:00,  1.24it/s]

..(dataset: Val) AUC 0.9507718853554332
..(dataset: Test) evaluation



100%|██████████| 56/56 [00:44<00:00,  1.26it/s]


..(dataset: Test) AUC 0.9496298813319404
